## Build Rules theo tài liệu

Dựa trên tài liệu y khoa chuẩn về phân độ bệnh tay chân miệng.

In [22]:
import pandas as pd
import numpy as np
import json

# ========================================
# ĐỊNH NGHĨA FEATURES THEO TÀI LIỆU
# ========================================

# Binary features (triệu chứng có/không)
BINARY_FEATURES = [
    # Độ 1
    "rash_hand_foot_mouth",      # phát ban tay chân miệng
    "mouth_ulcer",               # loét miệng
    
    # Độ 2a, 2b
    "startle_observed",          # giật mình ghi nhận lúc khám
    "seizure_history",           # co giật/ngủ gà
    "vomiting_many",             # nôn nhiều
    "lethargy",                  # lừ đừ
    "sleep_difficulty",          # khó ngủ
    "irritable",                 # quấy khóc vô cớ
    "antipyretic_no_response",   # sốt cao không đáp ứng hạ sốt
    
    # Độ 2b2
    "ataxia",                    # thất điều/run chi/loạng choạng
    "nystagmus_strabismus",      # rung giật nhãn cầu/lác mắt
    "limb_weakness",             # yếu/liệt chi
    "cranial_nerve_palsy",       # liệt thần kinh sọ (nuốt sặc/đổi giọng)
    "hypertonia",                # tăng trương lực cơ
    "avpu_P",                    # AVPU = P
    
    # Độ 3
    "resp_distress",             # thở nhanh/khó thở/rút lõm ức
    "mottled_sweat_cold",        # da nổi bông/vã mồ hôi/chi lạnh
    
    # Độ 4
    "apnea_irregular_breath",    # ngưng thở/rối loạn nhịp thở
    "cyanosis",                  # tím tái toàn thân
    "pulmonary_edema",           # phù phổi cấp
    "shock_pulse_absent",        # mạch không bắt được/HA không đo được
]

# Numeric features (giá trị đo lường)
NUMERIC_FEATURES = [
    "startle_per_30min",         # số lần giật mình/30 phút
    "fever_temp_c",              # nhiệt độ sốt (°C)
    "fever_days",                # số ngày sốt
    "hr_no_fever",               # mạch khi không sốt (lần/phút)
    "sbp",                       # huyết áp tâm thu (mmHg)
    "spo2",                      # SpO2 (%)
    "pulse_pressure",            # hiệu áp (mmHg)
    "gcs",                       # Glasgow Coma Scale
    "age_months",                # tuổi (tháng) - để tính ngưỡng HA
]

print(f"✓ Binary features: {len(BINARY_FEATURES)}")
print(f"✓ Numeric features: {len(NUMERIC_FEATURES)}")

✓ Binary features: 21
✓ Numeric features: 9


In [25]:
# ========================================
# TẠO RULES THEO TÀI LIỆU CHÍNH THỨC
# ========================================

all_rules = []

# ===== ĐỘ 1: Không biến chứng =====
# "Trẻ chỉ có phát ban tay chân miệng và/hoặc loét miệng"
# → "và/hoặc" = 3 rules: chỉ rash, chỉ ulcer, hoặc cả 2

all_rules.append({
    "id": "grade_1_rash",
    "name": "Độ 1 - Chỉ có phát ban tay chân miệng",
    "grade": "1",
    "priority": 0,
    "conditions": {
        "binary": {"rash_hand_foot_mouth": True},
        "numeric": {}
    }
})

all_rules.append({
    "id": "grade_1_ulcer",
    "name": "Độ 1 - Chỉ có loét miệng",
    "grade": "1",
    "priority": 0,
    "conditions": {
        "binary": {"mouth_ulcer": True},
        "numeric": {}
    }
})

# ===== ĐỘ 2A =====
# "Có ≥ 1 dấu hiệu sau:" → mỗi dấu hiệu là 1 rule riêng

# Rule 1: Giật mình < 2 lần/30 phút và không có lúc khám
all_rules.append({
    "id": "grade_2a_1",
    "name": "Độ 2a - Giật mình < 2 lần/30 phút (không ghi nhận lúc khám)",
    "grade": "2a",
    "priority": 1,
    "conditions": {
        "binary": {"startle_observed": False},
        "numeric": {"startle_per_30min": ("<", 2)}
    }
})

# Rule 2: Sốt ≥ 39°C
all_rules.append({
    "id": "grade_2a_2",
    "name": "Độ 2a - Sốt cao ≥ 39°C",
    "grade": "2a",
    "priority": 1,
    "conditions": {
        "binary": {},
        "numeric": {"fever_temp_c": (">=", 39)}
    }
})

# Rule 3-6: "Sốt > 2 ngày KÈM nôn nhiều/lừ đừ/khó ngủ/quấy khóc"
# "KÈM" = "và" → mỗi rule có 2 conditions
for symptom, name in [
    ("vomiting_many", "nôn nhiều"),
    ("lethargy", "lừ đừ"),
    ("sleep_difficulty", "khó ngủ"),
    ("irritable", "quấy khóc vô cớ")
]:
    all_rules.append({
        "id": f"grade_2a_{symptom}",
        "name": f"Độ 2a - Sốt > 2 ngày kèm {name}",
        "grade": "2a",
        "priority": 1,
        "conditions": {
            "binary": {symptom: True},
            "numeric": {"fever_days": (">", 2)}
        }
    })

# ===== ĐỘ 2B NHÓM 1 =====
# "Có ≥ 1 dấu hiệu sau:" → mỗi dấu hiệu là 1 rule riêng

# Rule 1: Giật mình ghi nhận lúc khám
all_rules.append({
    "id": "grade_2b1_1",
    "name": "Độ 2b1 - Giật mình ghi nhận lúc khám",
    "grade": "2b1",
    "priority": 2,
    "conditions": {
        "binary": {"startle_observed": True},
        "numeric": {}
    }
})

# Rule 2: Giật mình ≥ 2 lần/30 phút
all_rules.append({
    "id": "grade_2b1_2",
    "name": "Độ 2b1 - Giật mình ≥ 2 lần/30 phút",
    "grade": "2b1",
    "priority": 2,
    "conditions": {
        "binary": {},
        "numeric": {"startle_per_30min": (">=", 2)}
    }
})

# Rule 3: "Co giật kèm theo 1 dấu hiệu sau: + Ngủ gà + Mạch > 130"
# "Kèm theo 1 dấu hiệu sau" → cần seizure_history VÀ (ngủ gà HOẶC mạch > 130)
# Nhưng "ngủ gà" chính là seizure_history, nên có 2 rules:

all_rules.append({
    "id": "grade_2b1_3a",
    "name": "Độ 2b1 - Co giật (ngủ gà)",
    "grade": "2b1",
    "priority": 2,
    "conditions": {
        "binary": {"seizure_history": True},
        "numeric": {}
    }
})

all_rules.append({
    "id": "grade_2b1_3b",
    "name": "Độ 2b1 - Mạch > 130 lần/phút khi không sốt",
    "grade": "2b1",
    "priority": 2,
    "conditions": {
        "binary": {},
        "numeric": {"hr_no_fever": (">", 130)}
    }
})

# ===== ĐỘ 2B NHÓM 2 =====
# "Có ≥ 1 dấu hiệu sau:" → mỗi dấu hiệu là 1 rule riêng

# Nhóm 1: Sốt cao ≥ 39°C không đáp ứng hạ sốt
all_rules.append({
    "id": "grade_2b2_fever",
    "name": "Độ 2b2 - Sốt ≥ 39°C không đáp ứng hạ sốt",
    "grade": "2b2",
    "priority": 3,
    "conditions": {
        "binary": {"antipyretic_no_response": True},
        "numeric": {"fever_temp_c": (">=", 39)}
    }
})

# Mạch nhanh > 150
all_rules.append({
    "id": "grade_2b2_hr150",
    "name": "Độ 2b2 - Mạch > 150 lần/phút khi không sốt",
    "grade": "2b2",
    "priority": 3,
    "conditions": {
        "binary": {},
        "numeric": {"hr_no_fever": (">", 150)}
    }
})

# Các triệu chứng thần kinh ("hoặc" → mỗi cái 1 rule)
symptoms_2b2 = [
    ("ataxia", "Thất điều: run chi, run người, đi loạng choạng"),
    ("nystagmus_strabismus", "Rung giật nhãn cầu, lác mắt"),
    ("limb_weakness", "Yếu chi hoặc liệt chi"),
    ("cranial_nerve_palsy", "Liệt thần kinh sọ: nuốt sặc, thay đổi giọng nói"),
    ("hypertonia", "Tăng trương lực cơ"),
]

for symptom, name in symptoms_2b2:
    all_rules.append({
        "id": f"grade_2b2_{symptom}",
        "name": f"Độ 2b2 - {name}",
        "grade": "2b2",
        "priority": 3,
        "conditions": {
            "binary": {symptom: True},
            "numeric": {}
        }
    })

# Glasgow < 10 hoặc AVPU = P
all_rules.append({
    "id": "grade_2b2_gcs",
    "name": "Độ 2b2 - Glasgow Coma Scale < 10",
    "grade": "2b2",
    "priority": 3,
    "conditions": {
        "binary": {},
        "numeric": {"gcs": ("<", 10)}
    }
})

all_rules.append({
    "id": "grade_2b2_avpu",
    "name": "Độ 2b2 - AVPU = P",
    "grade": "2b2",
    "priority": 3,
    "conditions": {
        "binary": {"avpu_P": True},
        "numeric": {}
    }
})

print(f"✓ Độ 1: {len([r for r in all_rules if r['grade'] == '1'])} rules")
print(f"✓ Độ 2a: {len([r for r in all_rules if r['grade'] == '2a'])} rules")
print(f"✓ Độ 2b1: {len([r for r in all_rules if r['grade'] == '2b1'])} rules")
print(f"✓ Độ 2b2: {len([r for r in all_rules if r['grade'] == '2b2'])} rules")

✓ Độ 1: 2 rules
✓ Độ 2a: 6 rules
✓ Độ 2b1: 4 rules
✓ Độ 2b2: 9 rules


In [26]:
# ===== ĐỘ 3: Rối loạn thần kinh thực vật nặng =====
# "Có ≥ 1 dấu hiệu sau:" → mỗi dấu hiệu là 1 rule riêng

# Mạch > 170 khi không sốt
all_rules.append({
    "id": "grade_3_hr170",
    "name": "Độ 3 - Mạch > 170 lần/phút khi không sốt",
    "grade": "3",
    "priority": 4,
    "conditions": {
        "binary": {},
        "numeric": {"hr_no_fever": (">", 170)}
    }
})

# Tăng HA tâm thu (theo tuổi) - mỗi ngưỡng là 1 rule
for age_range, threshold, desc in [
    ((0, 12), 100, "Trẻ < 12 tháng"),
    ((12, 24), 110, "Trẻ 12-24 tháng"),
    ((24, 999), 115, "Trẻ ≥ 24 tháng")
]:
    all_rules.append({
        "id": f"grade_3_sbp_{threshold}",
        "name": f"Độ 3 - HA tâm thu ≥ {threshold} mmHg ({desc})",
        "grade": "3",
        "priority": 4,
        "conditions": {
            "binary": {},
            "numeric": {"sbp": (">=", threshold)}
        }
    })

# "Thở nhanh, khó thở, thở rít hoặc SpO2 < 94%"
# "hoặc" → 2 rules riêng

all_rules.append({
    "id": "grade_3_resp",
    "name": "Độ 3 - Thở nhanh/khó thở/thở rít",
    "grade": "3",
    "priority": 4,
    "conditions": {
        "binary": {"resp_distress": True},
        "numeric": {}
    }
})

all_rules.append({
    "id": "grade_3_spo2",
    "name": "Độ 3 - SpO2 < 94%",
    "grade": "3",
    "priority": 4,
    "conditions": {
        "binary": {},
        "numeric": {"spo2": ("<", 94)}
    }
})

# Da nổi bông (vân tím), vã mồ hôi, chi lạnh
all_rules.append({
    "id": "grade_3_mottled",
    "name": "Độ 3 - Da nổi bông (vân tím), vã mồ hôi, chi lạnh",
    "grade": "3",
    "priority": 4,
    "conditions": {
        "binary": {"mottled_sweat_cold": True},
        "numeric": {}
    }
})

# ===== ĐỘ 4: Suy hô hấp tuần hoàn nặng =====
# "Trẻ có ≥ 1 dấu hiệu sau:" → mỗi dấu hiệu là 1 rule riêng

# "Ngưng thở, rối loạn nhịp thở"
all_rules.append({
    "id": "grade_4_apnea",
    "name": "Độ 4 - Ngưng thở, rối loạn nhịp thở",
    "grade": "4",
    "priority": 5,
    "conditions": {
        "binary": {"apnea_irregular_breath": True},
        "numeric": {}
    }
})

# "Tím tái hoặc SpO2 < 92%"
# "hoặc" → 2 rules

all_rules.append({
    "id": "grade_4_cyanosis",
    "name": "Độ 4 - Tím tái toàn thân",
    "grade": "4",
    "priority": 5,
    "conditions": {
        "binary": {"cyanosis": True},
        "numeric": {}
    }
})

all_rules.append({
    "id": "grade_4_spo2",
    "name": "Độ 4 - SpO2 < 92%",
    "grade": "4",
    "priority": 5,
    "conditions": {
        "binary": {},
        "numeric": {"spo2": ("<", 92)}
    }
})

# Phù phổi cấp
all_rules.append({
    "id": "grade_4_pulmonary_edema",
    "name": "Độ 4 - Phù phổi cấp",
    "grade": "4",
    "priority": 5,
    "conditions": {
        "binary": {"pulmonary_edema": True},
        "numeric": {}
    }
})

# "Sốc khi trẻ có 1 trong 3 tiêu chuẩn sau:"
# "1 trong 3" → 3 rules riêng

# Tiêu chuẩn 1: Mạch không bắt được, HA không đo được
all_rules.append({
    "id": "grade_4_shock_1",
    "name": "Độ 4 - Sốc: Mạch không bắt được, HA không đo được",
    "grade": "4",
    "priority": 5,
    "conditions": {
        "binary": {"shock_pulse_absent": True},
        "numeric": {}
    }
})

# Tiêu chuẩn 2: Tụt HA - HA tâm thu (theo tuổi)
# "< 70mmHg (trẻ < 12 tháng), < 80mmHg (trẻ > 12 tháng)"
for age_range, threshold, desc in [
    ((0, 12), 70, "Trẻ < 12 tháng"),
    ((12, 999), 80, "Trẻ ≥ 12 tháng")
]:
    all_rules.append({
        "id": f"grade_4_shock_sbp_{threshold}",
        "name": f"Độ 4 - Sốc: HA tâm thu < {threshold} mmHg ({desc})",
        "grade": "4",
        "priority": 5,
        "conditions": {
            "binary": {},
            "numeric": {"sbp": ("<", threshold)}
        }
    })

# Tiêu chuẩn 3: HA kẹp - hiệu áp ≤ 25mmHg
all_rules.append({
    "id": "grade_4_shock_pp",
    "name": "Độ 4 - Sốc: Hiệu áp ≤ 25 mmHg",
    "grade": "4",
    "priority": 5,
    "conditions": {
        "binary": {},
        "numeric": {"pulse_pressure": ("<=", 25)}
    }
})

print(f"✓ Độ 3: {len([r for r in all_rules if r['grade'] == '3'])} rules")
print(f"✓ Độ 4: {len([r for r in all_rules if r['grade'] == '4'])} rules")
print(f"\n📊 TỔNG: {len(all_rules)} rules")

✓ Độ 3: 7 rules
✓ Độ 4: 8 rules

📊 TỔNG: 36 rules


In [27]:
# ========================================
# CHUYỂN ĐỔI SANG RULES.JSON
# ========================================

def convert_to_rules_json(rules_list):
    """Chuyển đổi từ format internal sang format rules.json"""
    rules_json = {
        "conclusion_rules": [],
        "intermediate_rules": []
    }
    
    for rule in rules_list:
        json_rule = {
            "id": rule["id"],
            "name": rule["name"],
            "priority": rule["priority"],
            "conditions": [],
            "conclusion": {
                "disease_level": rule["grade"]
            }
        }
        
        # Thêm binary conditions
        for field, value in rule["conditions"]["binary"].items():
            json_rule["conditions"].append({
                "field": field,
                "operator": "==",
                "value": value
            })
        
        # Thêm numeric conditions
        for field, (operator, threshold) in rule["conditions"]["numeric"].items():
            json_rule["conditions"].append({
                "field": field,
                "operator": operator,
                "value": threshold
            })
        
        rules_json["conclusion_rules"].append(json_rule)
    
    return rules_json

# Convert
final_rules_json = convert_to_rules_json(all_rules)

# Thống kê
print("=" * 50)
print("RULES.JSON MỚI - THEO TÀI LIỆU CHÍNH THỨC")
print("=" * 50)

from collections import Counter
grades = [r["conclusion"]["disease_level"] for r in final_rules_json["conclusion_rules"]]
grade_counts = Counter(grades)

for grade in sorted(grade_counts.keys()):
    print(f"  Độ {grade}: {grade_counts[grade]} rules")

print(f"\n  📊 Tổng: {len(final_rules_json['conclusion_rules'])} rules")

# Lưu file
output_path = r".\data\rules.json"
with open(output_path, 'w', encoding='utf-8') as f:
    json.dump(final_rules_json, f, ensure_ascii=False, indent=2)

print(f"\n💾 Đã lưu: {output_path}")

RULES.JSON MỚI - THEO TÀI LIỆU CHÍNH THỨC
  Độ 1: 2 rules
  Độ 2a: 6 rules
  Độ 2b1: 4 rules
  Độ 2b2: 9 rules
  Độ 3: 7 rules
  Độ 4: 8 rules

  📊 Tổng: 36 rules

💾 Đã lưu: .\data\rules.json
